# 1. Veri Setinin Yüklenmesi ve Düzenlenmesi

Veri seti oluşturulurken wikipedi, memorial ve medicana hastanelerinin sitelerinden faydalanılmıştır. 

Yapay veri ürettiğimiz kısımda kombinasyon işlemi gerçekleştirilmektedir. Bu kombinasyon işlemi semptom sayısı $n$ olmak üzere $2^n$ formülüyle hesaplanmaktadır. Bu da her bir hastalık için semtpom sayısı arttıkça kombinasyonların da üstel bir şekilde artmasına neden olmaktadır. 

Örneğin 10 semptom içeren hastalık için yapay verimizde 1024 satır oluşturulurken 15 semptom içeren hastalık için 32.768 satır oluşturulacaktır. Bu hem veri oluşturulurken gecikmeye yol açacak hem de verinin dengesiz (imbalanced) olmasına neden olacaktır. Bunun önüne geçmek için semptom sayısı maksimum 10 olarak sınırlandırılmıştır. 

Bilgiler yukarıda bahsedilen kaynaklardan alınırken semtpom sayısını azaltmak için aşağıdaki bahsedilen değişimler yapılmıştır. 

1. Yorgunluk, halsizlik, güçsüzlük neredeyse tüm hastalıklarda bulunduğundan semtpom olarak veri setine dahil edilmemiştir. 

2. Cillte sarılık, sarı cilt, cildin sararması gibi aynı anlam içeren semptomlar veri setine tek bir semptom olarak eklenmiştir. Tahmin sürecinde semptomların ortak bir isim altında birleştirilmesi kritik bir süreçken chatbot kısmında hastaların aynı semptomu farklı şekillerde belirtebilmeleri de çözüm gerektiren ayrı bir süreçtir. Bu sürece sonraki aşamalarda değinilecektir. 

3. Semptom isimlendirilmesinde ortak noktalar yaratılmaya çalışılmıştır. **Mide** **şiş**mesi, **mide**nin **şiş**mesi, **şiş**kin **mide** gibi ifadeler yerine şiş mide kullanılmıştır. Bu isimlendirme diğer organ veya bölgeler için de geçerli olmuştur. Böylece hastalar farklı şekillerde bu rahatsızlıklarını belirtseler de şiş ve mide bu belirtmelerin içinde yer aldığından tespit daha kolay yapılabilmektedir. 





In [32]:
import pandas as pd
from itertools import combinations

In [33]:
df = pd.read_csv('hastalik_semptom.csv', sep=";")

In [34]:
df.head()

,hastalik_ismi,semptom_1,semptom_2,semptom_3,semptom_4,semptom_5,semptom_6,semptom_7,semptom_8,semptom_9,semptom_10
0,addison_hastalığı,karın ağrısı,kilo kaybı,koyu cilt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ağız_kanseri,iyileşmeyen ağız yarası,ağız içinde yumru,boyunda yumru,gevşek diş,yutma güçlüğü,ağız kokusu,ağızda uyuşma,NaN,NaN,NaN
2,akciğer_kanseri,nefes darlığı,hırıltı,öksürük,kanlı balgam,iştahsızlık,göğüs ağrısı,kısık ses,yutma güçlüğü,NaN,NaN
3,akdeniz_anemisi,kansızlık,düzensiz kalp ritmi,nefes darlığı,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alerji,gözde kızarıklık,kaşıntı,ciltte döküntü,burun akıntısı,nefes darlığı,şişkinlik,hapşırma,NaN,NaN,NaN


Veri setini makine öğrenme algoritmamızda kullanabilmemiz için sütun verileri semptomların isimlerini içermelidir bu nedenle her satır değeri satırda belirtilen hastalık için o semptomun görülüp görülmediği (1: semptom var, 0: semptom yok) şeklinde düzenlenmelidir.

In [35]:
df.set_index("hastalik_ismi", drop=True, inplace=True)
df_to_dict = df.T.apply(lambda x: x.dropna().to_list())
df_organized = pd.DataFrame(list(df_to_dict.items()), columns=['Hastalık','Semptom'])

In [37]:
veri=pd.DataFrame()
for x in range(0, len(df_organized['Semptom'])):
    dm = pd.get_dummies(df_organized['Semptom'][x])
    dm.insert(0,"Hastalık",df_organized['Hastalık'][x], True)
    veri = pd.concat([dm.groupby(['Hastalık']).sum(), veri]).fillna(0)

In [39]:
veri.sort_index().head()

,baş ağrısı,bulanık görme,burun kanaması,düzensiz kalp ritmi,göğüs ağrısı,göğüste baskı,idrarda kan,nefes darlığı,ciltte kırmızlık,kaşıntı,...,öğün atlamak,şiş boğaz,hafıza kaybı,ruh hali değişimleri,kanlı balgam,ağız içinde yumru,ağızda uyuşma,gevşek diş,iyileşmeyen ağız yarası,koyu cilt
Hastalık,,,,,,,,,,,,,,,,,,,,,
addison_hastalığı,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
akciğer_kanseri,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
akdeniz_anemisi,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alerji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alzheimer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2 Yapay Veri Üretilmesi

Herhangi bir hastalık düşünüldüğünde bu hastalığa sahip kişi tüm semptomları göstermeyebilir. Ayrıca aynı hastalıkta farklı kombinasyonlarda semptomlar gözlemlenebilir. Bu bölümde bir hastane verisini taklit ediyoruz. Örneğin bir hastalıkta 7 semptom bulunuyorsa $2^7$ adet veri oluşturulabilir. 

In [40]:
semptomlar = veri.columns.to_list()

In [41]:
veri_komb = pd.DataFrame(columns=semptomlar)

In [43]:
for key, values in df_to_dict.items():
    # Populate rows for combination dataset
    for comb in range(1, len(values)+1):
        for subset in combinations(values, comb):
            row_comb = dict({x:0 for x in semptomlar})
            for sym in list(subset):
                row_comb[sym]=1
            row_comb['hastalik']=key
            veri_komb = veri_komb.append(pd.Series(row_comb), ignore_index=True)

In [44]:
len(veri_komb)

6920

6920 örnek içeren bu veriyi üretmek süre aldığından sonraki çalışmalarda  kullanmak için csv tipinde bir dosya şeklinde kaydediyoruz.

In [45]:
veri_komb.to_csv("veri_komb.csv", index=None, encoding='iso-8859-9')